In [48]:
import importlib

import numpy as np
import pandas as pd
import numpy
import lab_graph

importlib.reload(lab_graph)

#Fix colormath depreciated numpy method
def patch_asscalar(a):
    return a.item()
setattr(numpy, "asscalar", patch_asscalar)

df = pd.read_csv("data/all-colors-unique.csv")

#Create a 2D array of Lab values
lab_list = list(df["L*a*b Value"])
lab_list = [x.split(", ") for x in lab_list]
lab_points = [[float(x) for x in row] for row in lab_list]

lab_graph.generate_lab_3d_all_colors(df, "All colors", "N/a")

In [24]:
from scipy.spatial import ConvexHull

hull = ConvexHull(points=lab_points)

hull.volume

476503.15507333336

In [42]:
convex_df = df.iloc[hull.vertices].copy()

lab_graph.generate_lab_3d_all_colors(convex_df, "Convex Hull of Points", "N/a")

In [62]:
from scipy.spatial.distance import pdist, squareform

euclidean_distances = squareform(pdist(lab_points))

convex_hull_score = lab_graph.score(euclidean_distances, list(hull.vertices))
print(convex_hull_score)

82.85095504753694


## Convex Hull Scores

21 Colors: 82.8510 (Slightly outperforms the greedy search)

# K-Means Clustering

In [53]:
from scipy.cluster.vq import kmeans

centroids = kmeans(lab_points, 16)

In [54]:
close_to_centroids = []

from scipy.spatial import distance
closest_indices = []
for centroid in centroids[0]:

    current_distances = []

    #For each color in the space, compute the distance
    for color in lab_points:
        current_distances += [distance.euclidean(centroid, color)]

    #The index of the minimum distance is the color we want
    closest = min(current_distances)
    closest_index = current_distances.index(closest)
    closest_indices += [closest_index]

len(closest_indices)

16

In [55]:
centroid_df = df.iloc[closest_indices].copy()

lab_graph.generate_lab_3d_all_colors(centroid_df, "K-Means Points", "N/a")

In [58]:

# kmeans_selected = [lab_points[i] for i in closest_indices]
# print(kmeans_selected)
score = lab_graph.score(euclidean_distances, closest_indices)
print(score)

56.874489346286325


## K means Scores

Score for 24 Points: 62.5369
Score for 16 Points: 56.8744